In [1]:
import pandas as pd
import numpy as np
from pystoned import CNLS, CNLSDDF, StoNED
from pystoned.constant import CET_MULT, CET_ADDI, FUN_COST, FUN_PROD, RTS_VRS, RTS_CRS, RED_MOM, OPT_LOCAL, RED_QLE, RED_KDE

In [2]:
data = pd.read_csv("../../data_ready/data_ready.csv").replace([np.inf, -np.inf], np.nan).dropna()

In [3]:
data_hun = data[data["CNT"] == 'HUN']

inputs_hun = data_hun[["total_students", "ratio_f2m", "ratio_s2t", "ratio_ft2pt"]].values.tolist()
outputs_hun = data_hun[["avg_math", "avg_science", "avg_read"]]
d_outputs_hun = data_hun[["avg_math", "avg_science", "avg_read"]].values.tolist()
undesirables_hun = data_hun[["dropout_rate"]].values.tolist()
contextuals_hun = data_hun[["bol_competition", "bol_location"]].values.tolist()
data_hun["avg_scores"] = outputs_hun.mean(axis = 1)
z_output_hun = data_hun[["avg_scores"]].values.tolist()

<ipython-input-3-99867c94995f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_hun["avg_scores"] = outputs_hun.mean(axis = 1)


In [4]:
model1_hun = CNLS.CNLS(
    y = z_output_hun, 
    x = inputs_hun, 
    z = contextuals_hun, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_CRS
)

model2_hun = CNLS.CNLS(
    y = z_output_hun, 
    x = inputs_hun, 
    z = contextuals_hun, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_VRS
)

# define the CNLSDDF model with undesirable output
model3_hun = CNLSDDF.CNLSDDF(
    y = d_outputs_hun, 
    x = inputs_hun, 
    b = undesirables_hun, 
    gb = -1.0,
    gy = [1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4_hun = CNLSDDF.CNLSDDF(
    y = d_outputs_hun, 
    x = inputs_hun, 
    gy = [1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

In [5]:
model1_hun.optimize(OPT_LOCAL)
model2_hun.optimize(OPT_LOCAL)
model3_hun.optimize(OPT_LOCAL)
model4_hun.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 9801            
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 695             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.01    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 99
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.0

In [6]:
### inefficiency
ineff1_hun = StoNED.StoNED(model1_hun)
ineff2_hun = StoNED.StoNED(model2_hun)
ineff3_hun = StoNED.StoNED(model3_hun)
ineff4_hun = StoNED.StoNED(model4_hun)

In [7]:
res_hun = {
    "res1_hun" : {
        "alpha" : "None",
        "beta" : model1_hun.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model1_hun.get_lamda(),
        "residuals" : model1_hun.get_residual(),
        "frontier" : model1_hun.get_frontier(),
        "unconditional_expected_inefficiency" : ineff1_hun.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff1_hun.get_technical_inefficiency(method = RED_QLE)
    },


    "res2_hun" : {
        "alpha" : model2_hun.get_alpha(),
        "beta" : model2_hun.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model2_hun.get_lamda(),
        "residuals" : model2_hun.get_residual(),
        "frontier" : model2_hun.get_frontier(),
        "unconditional_expected_inefficiency" : ineff2_hun.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff2_hun.get_technical_inefficiency(method = RED_QLE)
    },


    "res3_hun" : {
        "alpha" : model3_hun.get_alpha(),
        "beta" : model3_hun.get_beta(),
        "delta" : model3_hun.get_delta(),
        "gamma" : model3_hun.get_gamma(),
        "lamda" : "None",
        "residuals" : model3_hun.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff3_hun.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff3_hun.get_technical_inefficiency(method = RED_QLE)
    },


    "res4_hun" : {
        "alpha" : "None",
        "beta" : model4_hun.get_beta(),
        "delta" : "None",
        "gamma" : model4_hun.get_gamma(),
        "lamda" : "None",
        "residuals" : model4_hun.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff4_hun.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff4_hun.get_technical_inefficiency(method = RED_QLE)
    }
}


In [8]:
import pickle
file = 'res_hun'
open_file = open(file, "wb")

pickle.dump(res_hun, open_file)
open_file.close()


# open_file = open(file, "rb")
# res_hun = pickle.load(open_file)
# open_file.close()

